In [1]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/item_feature_python_extract_Training_all.csv', encoding = 'utf8')

<ipython-input-4-c3c8f57a4438>:1: DtypeWarning: Columns (10,12,15,19,20,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/item_feature_python_extract_Training_all.csv', encoding = 'utf8')


In [4]:
# List of unique accessions
unique_accessions = ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']



# Dictionary to store the dataframes
dfs = {}

# Loop through the unique accessions
for accession in unique_accessions:
    # Create the dataframe name
    path = '/content/drive/MyDrive/NAEP_Comp/'
    df_name = 'df_cleaned' + accession

    # Read the CSV file into a dataframe
    df = pd.read_csv(path + df_name + '.csv')

    # Add the dataframe to the dictionary
    dfs[accession] = df

<ipython-input-4-87736d8d5c62>:17: DtypeWarning: Columns (29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + df_name + '.csv')


In [5]:
df = dfs['VH271613']

In [ ]:
df = df[["student_id", "predict_from", "score_to_predict"]]

In [ ]:
def detect_minority_majority_classes(df, label_column):
    """
    Detects the minority and majority classes in a DataFrame based on a specified label column.

    Args:
        df (pandas.DataFrame): The DataFrame containing the label column.
        label_column (str): The name of the column containing the class labels.

    Returns:
        minority_classes (list): List of detected minority class labels.
        majority_class (str): Detected majority class label.
    """
    class_counts = df[label_column].value_counts()
    minority_classes = class_counts[class_counts < class_counts.max()].index.tolist()
    majority_class = class_counts.idxmax()
    return minority_classes, majority_class

In [ ]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nlpaug.augmenter.word as naw

import pandas as pd
import nlpaug.augmenter.word as naw

def augment_minority_class_text(df, text_column, label_column, minority_classes, majority_class):
    """
    Augments the text of the minority classes in a DataFrame using NLPAug library.

    Args:
        df (pandas.DataFrame): The DataFrame containing the text and label columns.
        text_column (str): The name of the column containing the text.
        label_column (str): The name of the column containing the class labels.
        minority_classes (list): List of minority class labels.
        majority_class (str): The majority class label.

    Returns:
        augmented_df (pandas.DataFrame): The augmented DataFrame.
    """
    augmented_texts = []
    aug = naw.RandomWordAug(action="swap")

    for minority_class in minority_classes:
        # Filter the dataframe to get only the minority class rows
        minority_df = df[df[label_column] == minority_class]
        majority_df = df[df[label_column] == majority_class]
        minority_count = len(minority_df)
        majority_count = len(majority_df)

        # Check if augmentation is required based on class imbalance
        if minority_count >= 0.7 * majority_count:
            continue

        # Calculate the number of augmentations required
        num_augmentations = int(0.7 * majority_count) - minority_count

        # Augment the text of the minority class
        while num_augmentations > 0:
            for text in minority_df[text_column]:
                augmented_text = aug.augment(text)
                augmented_texts.append(augmented_text)
                num_augmentations -= 1
                if num_augmentations == 0:
                    break

        # Create a new dataframe with augmented texts
        augmented_df = pd.DataFrame({text_column: augmented_texts})
        augmented_df[label_column] = minority_class
        print("Value Counts = " + str(augmented_df[label_column].value_counts()))
        # Concatenate the augmented dataframe with the original dataframe
        augmented_df = pd.concat([df, augmented_df], ignore_index=True)

    return augmented_df



In [ ]:
smote_names = ['VH266015','VH266510','VH269384','VH271613','VH302907''VH507804''VH525628']

In [ ]:

for prompts in unique_accessions :
    df = dfs[str(prompts)]
    #print(df['assigned_score'].value_counts())

    if prompts in smote_names :
        print(prompts)

        minority_class, majority_class = detect_minority_majority_classes(df, 'assigned_score')
        df = augment_minority_class_text(df,'predict_from', 'assigned_score', minority_class, majority_class )
        dfs[str(prompts)] = df


VH266015
Value Counts = 3    3710
Name: assigned_score, dtype: int64
Value Counts = 4    8758
Name: assigned_score, dtype: int64
VH266510
Value Counts = 3    13342
Name: assigned_score, dtype: int64
Value Counts = 2    33186
Name: assigned_score, dtype: int64
VH269384
Value Counts = 4    3924
Name: assigned_score, dtype: int64
Value Counts = 3    8240
Name: assigned_score, dtype: int64
VH271613
Value Counts = 2    168
Name: assigned_score, dtype: int64
Value Counts = 4    10708
Name: assigned_score, dtype: int64
Value Counts = 3    21372
Name: assigned_score, dtype: int64


In [ ]:
def extract_first_string(value):
    if isinstance(value, list):
        if len(value) > 0 and isinstance(value[0], str):
            return value[0]
    return value

In [ ]:
for i, df in enumerate(dfs):
    name = unique_accessions[i]
    df = dfs[name]
    df['predict_from'] = df['predict_from'].apply(extract_first_string)


In [6]:
df = dfs['VH271613']

In [7]:
df = df[["student_id", "predict_from", "score_to_predict"]]

In [8]:
from sklearn.model_selection import train_test_split

In [52]:
df_train, df_test = train_test_split(df, test_size = 0.15, stratify = df['score_to_predict'], random_state=11)

In [10]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.3 MB/s eta 0:00:00


In [16]:
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('tbs17/MathBERT', do_lower_case=True)

In [17]:
from transformers import BertModel
import torch.nn as nn

class BertForSentenceClassification(BertModel):
    def __init__(self, config):
        super().__init__(config)

        ##### START YOUR CODE HERE #####
        # Add a linear classifier that map BERTs [CLS] token representation to the unnormalized
        # output probabilities for each class (logits).
        # Notes:
        #  * See the documentation for torch.nn.Linear
        #  * You do not need to add a softmax, as this is included in the loss function
        #  * The size of BERTs token representation can be accessed at config.hidden_size
        #  * The number of output classes can be accessed at config.num_labels
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        ##### END YOUR CODE HERE #####
        self.loss = torch.nn.CrossEntropyLoss()

    def forward(self, labels=None, **kwargs):
        outputs = super().forward(**kwargs)
        ##### START YOUR CODE HERE #####
        # Pass BERTs [CLS] token representation to this new classifier to produce the logits.
        # Notes:
        #  * The [CLS] token representation can be accessed at outputs.pooler_output
        cls_token_repr = outputs.pooler_output
        print(outputs.pooler_output.shape)
        logits = self.classifier(cls_token_repr)
        ##### END YOUR CODE HERE #####
        if labels is not None:
            outputs = (logits, self.loss(logits, labels))
        else:
            outputs = (logits,)
        return outputs

In [53]:
from torch.optim import AdamW
import time
import datetime
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import numpy as np
from sklearn.metrics import cohen_kappa_score


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def get_optimizer_and_scheduler(model, total_steps, lr=2e-5, weight_decay=0.01):
    # Apply weight decay to all parameters beside the biases or LayerNorm weights
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': weight_decay},
        {
            'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
        }
    ]
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        # Warmup learning rate for first 10% of training steps
        num_warmup_steps=int(0.10 * total_steps),
        num_training_steps=total_steps,
    )
    return optimizer, scheduler

def train_model(model, epochs, train_dataloader, validation_dataloader,test_indexs):
    # Use GPU, if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model = model.to(device)
    kappa_score_int = 0
    best_model = model

    # Setup optimizer and LR scheduler
    total_steps = len(train_dataloader) * epochs
    optimizer, scheduler = get_optimizer_and_scheduler(
        model, total_steps, lr=5e-5, weight_decay=0.01
    )

    loss_values = []
    eval_accs = []
        # Tracking variables


    for epoch in range(0, epochs):
        t0 = time.time()

        total_loss = 0
        model.train()
        predictions, true_labels = [], []

        with tqdm(train_dataloader, unit="batch") as train_pbar:
            for batch in train_pbar:
                train_pbar.set_description(f"Training (epoch {epoch + 1})")
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                model.zero_grad()

                # Perform a forward pass (evaluate the model on this training batch).
                # This will return the loss because we have provided the `labels`.
                outputs = model(
                    input_ids=b_input_ids,
                    attention_mask=b_input_mask,
                    labels=b_labels
                )

                # The call to `model` always returns a tuple, so we need to pull the
                # loss value out of the tuple.
                _, loss = outputs

                # Accumulate the training loss over all of the batches so that we can
                # calculate the average loss at the end. `loss` is a Tensor containing a
                # single value; the `.item()` function just returns the Python value
                # from the tensor.
                total_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Clip the norm of the gradients to 1.0.
                # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                optimizer.step()

                # Update the learning rate.
                scheduler.step()

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)

        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)

        print("  * Average training loss: {0:.2f}".format(avg_train_loss))
        print("  * Training epoch took: {:}".format(format_time(time.time() - t0)))

        print("Running Validation...")

        t0 = time.time()
        model.eval()

        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            with torch.no_grad():
                # Forward pass, calculate logit predictions.
                # This will return the logits rather than the loss because we have
                # not provided labels.
                # token_type_ids is the same as the "segment ids", which
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                outputs = model(
                    input_ids=b_input_ids,
                    attention_mask=b_input_mask
                )

            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = outputs[0]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            # Calculate the accuracy for this batch of test sentences.
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy
            # Track the number of batches
            nb_eval_steps += 1
            predictions.append(logits)
            true_labels.append(label_ids)

        predictions = np.concatenate(predictions)
        true_labels = np.concatenate(true_labels)

        predicted_labels = np.argmax(predictions, axis=1)
        kappa_score = cohen_kappa_score(true_labels, predicted_labels, labels=None, weights= 'quadratic', sample_weight=None)
        if kappa_score > kappa_score_int :
          kappa_score_int = kappa_score
          best_model = model
          results_df = pd.DataFrame(index=test_indexs)
          results_df['indexes'] = test_indexs
          results_df['TrueValue'] = true_labels
          results_df['PredictedValue'] = predicted_labels



        avg_eval_acc = eval_accuracy/nb_eval_steps
        print("  * Accuracy: {0:.2f}".format(avg_eval_acc))
        print("  * Validation took: {:}".format(format_time(time.time() - t0)))
        print("Kappa Score = " + str(kappa_score))
        eval_accs.append(avg_eval_acc)
    print("Training complete!")
    return loss_values, eval_accs, results_df, kappa_score_int, best_model

In [54]:
df_test

,student_id,predict_from,score_to_predict
23253,bOo8q8Oj0N,you mtda,1
13828,qm76aAIl9D,PHIL IS OLDER THAN ZACH,1
21679,Si3MMasECV,becus it dont atted up,1
3270,CZVb4lFrLE,fulse,1
12513,lhFUAuLxPL,It is not true because in the,1
...,...,...,...
24442,huHHC6Z1gM,It is the same,1
25909,pdujpF3gGD,because zach is 6 years older then tim.,1
23723,dwlDM7JMK8,this statement wouldn't be true becase say Phi...,3
11821,j1VnE6sVDd,It will not be true because Phil will be a dif...,1


In [65]:
def datapreprocess(df):
    bert_tokenizer = AutoTokenizer.from_pretrained('tbs17/MathBERT', do_lower_case=True)
    train_inputs = df_train.predict_from.values
    train_labels = df_train.score_to_predict.values

    test_inputs = df_test.predict_from.values
    test_labels = df_test.score_to_predict.values
    test_indexes = df_test.student_id
    train_inputs = ["[CLS] " + text + " [SEP]" for text in train_inputs]
    test_inputs = ["[CLS] " + text + " [SEP]" for text in test_inputs]
    tokenized_inputs_train = bert_tokenizer(
        train_inputs,padding = True, max_length=120,truncation = True,
        return_tensors='pt',
    )

    tokenized_inputs_valid= bert_tokenizer(
        test_inputs,padding = True,max_length=120, truncation = True,
        return_tensors='pt',
    )


    train_inputs = tokenized_inputs_train['input_ids']
    train_masks = tokenized_inputs_train['attention_mask']

    test_inputs = tokenized_inputs_valid['input_ids']
    test_masks = tokenized_inputs_valid['attention_mask']

    return train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, test_indexes


In [57]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

def train_valid_split(train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, test_indexes, batch_size=32 ):
    # Use 80% for training and 20% for validation.

    train_labels = torch.tensor(train_labels)
    validation_labels = torch.tensor(test_labels)

    # Create the DataLoader for our training set.
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

    # Create the DataLoader for our validation set.
    validation_data = TensorDataset(test_inputs, test_masks, validation_labels)
    validation_dataloader = DataLoader(validation_data, shuffle=False, batch_size=batch_size)

    return train_dataloader, validation_dataloader

In [59]:
from transformers import BertModel
import torch.nn as nn

class BertCNNLSTMClassifier(BertModel):
    def __init__(self, config):
        super().__init__(config)

        self.lstm_hidden_size = 512
        self.lstm_num_layers = 2
        self.cnn_num_filters = 256
        self.cnn_kernel_size = 3

        # Add a linear classifier that maps the concatenated representation to the unnormalized
        # output probabilities for each class (logits).
        self.classifier = nn.Linear(config.hidden_size + self.lstm_hidden_size * 2 + self.cnn_num_filters, config.num_labels)

        # Bidirectional LSTM layer
        self.lstm = nn.LSTM(config.hidden_size, self.lstm_hidden_size, num_layers=self.lstm_num_layers, batch_first=True, bidirectional=True)

        # CNN layer
        self.cnn = nn.Conv1d(
            in_channels=self.lstm_hidden_size * 2,
            out_channels=self.cnn_num_filters,
            kernel_size=self.cnn_kernel_size,
            padding=(self.cnn_kernel_size - 1) // 2
        )

        # Activation function
        self.activation = nn.ReLU()

        # Loss function
        self.loss = torch.nn.CrossEntropyLoss()

    def forward(self, labels=None, **kwargs):
        outputs = super().forward(**kwargs)

        # Pass BERT's [CLS] token representation to the LSTM layer
        cls_token_repr = outputs.pooler_output
        lstm_output, _ = self.lstm(cls_token_repr.unsqueeze(1))

        # Reshape the LSTM output for CNN
        lstm_output = lstm_output.permute(0, 2, 1)  # Reshape for CNN

        # Pass the LSTM output through the CNN layer
        cnn_output = self.cnn(lstm_output)
        cnn_output = cnn_output.squeeze(2)  # Remove the last dimension

        # Reshape the LSTM output for concatenation
        lstm_output = lstm_output.squeeze(2)

        # Concatenate the [CLS] token representation, LSTM output, and CNN output
        concatenated_repr = torch.cat((cls_token_repr, lstm_output, cnn_output), dim=1)

        # Pass the concatenated representation to the linear classifier
        logits = self.classifier(concatenated_repr)

        if labels is not None:
            outputs = (logits, self.loss(logits, labels))
        else:
            outputs = (logits,)

        return outputs


In [60]:
def train(df, name) :
  train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, test_indexes = datapreprocess(df)
  train_dataloader,validation_dataloader= train_valid_split(train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, test_indexes )
  mathbert = BertCNNLSTMClassifier.from_pretrained(
    "tbs17/MathBERT",  # the name of the pretrained model
    num_labels=df['score_to_predict'].nunique()+1,  )
  mathbert_loss_vals, mathbert_eval_accs, results_df, kappa_score, model = train_model(model=mathbert,
    epochs=5,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader, test_indexs=test_indexes)
  #PATH1 = "/content/drive/MyDrive/NAEP_Comp/Bert+CNN/" + name  +".pt"
  #torch.save(mathbert, PATH1)
  #results_df.to_csv('/content/drive/MyDrive/NAEP_Comp/MathBert+CNN/' + str(name) + '.csv', index=False)
  return kappa_score, results_df, model
   # the number of classes in our downstream task



In [61]:
df

,student_id,predict_from,score_to_predict
0,00DX05DRh5,it is not true because phil's is 3 times yonge...,1
1,00N3yEYMAy,because is zach is 4 years older than tim that...,1
2,00TuatdAM2,because now zach will be older than tim and alex,1
3,00wXPqjAxM,"because we older ,not yonger",1
4,00wtajhH6E,because i,1
...,...,...,...
27843,zyUjt9xqkg,because phil's is 3 years older than alex's age,1
27844,zzBMPZ2ulP,because what if he was exp: he's like 16 and zacj,1
27845,zzEWXuRyL6,ijhgghhhhhhhhhhhjhu,1
27846,zzfwDRPzZC,it will not be true because zach will be older...,1


In [62]:
torch.cuda.empty_cache()

In [66]:
score, results_df, model = train(df, 'Test')

Some weights of the model checkpoint at tbs17/MathBERT were not used when initializing BertCNNLSTMClassifier: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertCNNLSTMClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertCNNLSTMClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda


Training (epoch 1): 100%|██████████| 740/740 [07:09<00:00,  1.72batch/s]


  * Average training loss: 0.14
  * Training epoch took: 0:07:09
Running Validation...
  * Accuracy: 0.97
  * Validation took: 0:00:25
Kappa Score = 0.7216831039685551


Training (epoch 2): 100%|██████████| 740/740 [07:08<00:00,  1.73batch/s]


...

  * Average training loss: 0.06
  * Training epoch took: 0:07:08
Running Validation...
  * Accuracy: 0.98
  * Validation took: 0:00:25
Kappa Score = 0.7708865895252405


Training (epoch 4): 100%|██████████| 740/740 [07:08<00:00,  1.73batch/s]


  * Average training loss: 0.04
  * Training epoch took: 0:07:09
Running Validation...
  * Accuracy: 0.98
  * Validation took: 0:00:25
Kappa Score = 0.7748096251421537


Training (epoch 5): 100%|██████████| 740/740 [07:08<00:00,  1.73batch/s]


  * Average training loss: 0.02
  * Training epoch took: 0:07:09
Running Validation...
  * Accuracy: 0.98
  * Validation took: 0:00:25
Kappa Score = 0.7897115793208438
Training complete!


In [76]:
results_df['student_id'] = results_df.index
results_df = results_df.rename(columns = {'TrueValue': 'true_labels','\tpredicted' : 'predicted'})
results_df

,indexes,true_labels,predicted,student_id
student_id,,,,
bOo8q8Oj0N,NaN,1,1,bOo8q8Oj0N
qm76aAIl9D,NaN,1,1,qm76aAIl9D
Si3MMasECV,NaN,1,1,Si3MMasECV
CZVb4lFrLE,NaN,1,1,CZVb4lFrLE
lhFUAuLxPL,NaN,1,1,lhFUAuLxPL
...,...,...,...,...
huHHC6Z1gM,NaN,1,1,huHHC6Z1gM
pdujpF3gGD,NaN,1,1,pdujpF3gGD
dwlDM7JMK8,NaN,3,3,dwlDM7JMK8


In [80]:
final_results = results_df[['true_labels','predicted']].copy()
final_results.reset_index(inplace = True)
final_results

,student_id,true_labels,predicted
0,bOo8q8Oj0N,1,1
1,qm76aAIl9D,1,1
2,Si3MMasECV,1,1
3,CZVb4lFrLE,1,1
4,lhFUAuLxPL,1,1
...,...,...,...
4173,huHHC6Z1gM,1,1
4174,pdujpF3gGD,1,1
4175,dwlDM7JMK8,3,3
4176,j1VnE6sVDd,1,1


In [81]:
df_copy = dfs['VH271613'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]
df_copy

,student_id,score_to_predict,srace10,dsex,accom2,iep,lep
0,00DX05DRh5,1,3.0,2.0,2.0,2.0,2.0
1,00N3yEYMAy,1,1.0,2.0,2.0,2.0,2.0
2,00TuatdAM2,1,1.0,2.0,2.0,2.0,2.0
3,00wXPqjAxM,1,7.0,2.0,2.0,2.0,2.0
4,00wtajhH6E,1,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...
27843,zyUjt9xqkg,1,3.0,1.0,1.0,2.0,1.0
27844,zzBMPZ2ulP,1,3.0,1.0,2.0,2.0,1.0
27845,zzEWXuRyL6,1,1.0,1.0,2.0,2.0,2.0
27846,zzfwDRPzZC,1,2.0,1.0,2.0,2.0,2.0


In [84]:
merged_df = final_results.merge(df_copy, on="student_id", how="inner")

merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH271613_mathbert.csv', index = False)
merged_df

,student_id,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,bOo8q8Oj0N,1,1,1,3.0,1.0,1.0,1.0,1.0
1,qm76aAIl9D,1,1,1,1.0,1.0,2.0,1.0,2.0
2,Si3MMasECV,1,1,1,3.0,2.0,1.0,1.0,2.0
3,CZVb4lFrLE,1,1,1,3.0,2.0,2.0,2.0,2.0
4,lhFUAuLxPL,1,1,1,1.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
4173,huHHC6Z1gM,1,1,1,1.0,1.0,2.0,2.0,2.0
4174,pdujpF3gGD,1,1,1,1.0,1.0,1.0,1.0,2.0
4175,dwlDM7JMK8,3,3,3,1.0,2.0,2.0,2.0,2.0
4176,j1VnE6sVDd,1,1,1,1.0,2.0,2.0,2.0,2.0


In [86]:

df_test = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/df_test.csv')
dfs_test = {}
# Unique values in the 'accession' column
unique_accessions = df_test['accession'].unique().tolist()
# Loop through the unique accessions
for accession in unique_accessions:
    # Filter the dataframe for the current accession
    grouped_df = df_test[df_test['accession'] == accession]
    # Create the dataframe name
    path = '/content/drive/MyDrive/NAEP_Comp/Final Results'
    df_name = 'df_test' + accession
    dfs_test[accession] = grouped_df

    # Save the dataframe as a CSV file
    #grouped_df.to_csv(path + df_name + '.csv', index=False)

<ipython-input-86-f97152282f9b>:1: DtypeWarning: Columns (20,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/df_test.csv')


In [102]:
df_test = dfs_test['VH271613']

df_test

,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
14247,00u9rrqZuh,VH271613,NaN,it seys in the tope Phil's age will be 3 times...,2017,3,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c(""1"", """")",list()
14248,02G1Gxbmhf,VH271613,NaN,zach is younger than Alexs is 6time older than...,2017,2,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
14249,053OYb9O4y,VH271613,NaN,i dont know,2017,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c("""", ""2"")",list()
14250,05M1oLdPvU,VH271613,NaN,cause it did not tell you their age,2017,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""1"")","c(NA, 2)",list()
14251,06e1PZnECl,VH271613,NaN,THE STATEMENT IS NOT TRUE BECAUSE PHIL IS 3 TI...,2017,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17336,zrKGO3SIy8,VH271613,NaN,56,2019,2,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 1,TRUE FALSE,FALSE FALSE
17337,zutovXyxK3,VH271613,NaN,tim is six years younger then phile so that te...,2019,6,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 2,TRUE FALSE,FALSE FALSE
17338,zw8QXgpLnG,VH271613,NaN,Because the amount of how much older you are t...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE
17339,zyyahLMxtS,VH271613,NaN,Because Zach is already one year younger then ...,2019,7,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE


In [103]:
df_test["predict_from"]= df_test["predict_from"].fillna('N/A')

In [104]:
df_test

,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
14247,00u9rrqZuh,VH271613,NaN,it seys in the tope Phil's age will be 3 times...,2017,3,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c(""1"", """")",list()
14248,02G1Gxbmhf,VH271613,NaN,zach is younger than Alexs is 6time older than...,2017,2,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
14249,053OYb9O4y,VH271613,NaN,i dont know,2017,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c("""", ""2"")",list()
14250,05M1oLdPvU,VH271613,NaN,cause it did not tell you their age,2017,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""1"")","c(NA, 2)",list()
14251,06e1PZnECl,VH271613,NaN,THE STATEMENT IS NOT TRUE BECAUSE PHIL IS 3 TI...,2017,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17336,zrKGO3SIy8,VH271613,NaN,56,2019,2,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 1,TRUE FALSE,FALSE FALSE
17337,zutovXyxK3,VH271613,NaN,tim is six years younger then phile so that te...,2019,6,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 2,TRUE FALSE,FALSE FALSE
17338,zw8QXgpLnG,VH271613,NaN,Because the amount of how much older you are t...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE
17339,zyyahLMxtS,VH271613,NaN,Because Zach is already one year younger then ...,2019,7,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE


In [105]:
test_inputs1 = df_test.predict_from.values.tolist()

In [106]:
test_inputs = ["[CLS] " + text + " [SEP]" for text in test_inputs1]
tokenized_inputs_test = bert_tokenizer(test_inputs, add_special_tokens=True,
        padding='max_length',
        max_length=120,
        return_tensors='pt',
        truncation=True)

In [107]:
test_input_ids = tokenized_inputs_test['input_ids']
test_attention_masks = tokenized_inputs_test['attention_mask']

In [108]:
test_data = TensorDataset(test_input_ids, test_attention_masks)
test_loader = DataLoader(test_data, shuffle=True, batch_size=32)

In [109]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [113]:
t0 = time.time()
predictions = []
model.eval()
for batch in test_loader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch  # Remove the unused b_labels

    with torch.no_grad():
        outputs = model(input_ids=b_input_ids, attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)

predictions = np.concatenate(predictions)
predicted_labels = np.argmax(predictions, axis=1)
print(np.unique(predicted_labels))

[1 2 3]


In [114]:
df_test['score_to_predict'] = predicted_labels
df_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH271613.csv', index = False)
df_test

,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
14247,00u9rrqZuh,VH271613,1,it seys in the tope Phil's age will be 3 times...,2017,3,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c(""1"", """")",list()
14248,02G1Gxbmhf,VH271613,1,zach is younger than Alexs is 6time older than...,2017,2,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
14249,053OYb9O4y,VH271613,1,i dont know,2017,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c("""", ""2"")",list()
14250,05M1oLdPvU,VH271613,1,cause it did not tell you their age,2017,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""1"")","c(NA, 2)",list()
14251,06e1PZnECl,VH271613,1,THE STATEMENT IS NOT TRUE BECAUSE PHIL IS 3 TI...,2017,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17336,zrKGO3SIy8,VH271613,1,56,2019,2,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 1,TRUE FALSE,FALSE FALSE
17337,zutovXyxK3,VH271613,1,tim is six years younger then phile so that te...,2019,6,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 2,TRUE FALSE,FALSE FALSE
17338,zw8QXgpLnG,VH271613,1,Because the amount of how much older you are t...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE
17339,zyyahLMxtS,VH271613,1,Because Zach is already one year younger then ...,2019,7,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE


In [ ]:
#results = {}
for i, df in enumerate(dfs, start = 7):
    name = unique_accessions[i]
    df = dfs[name]
    print(name)
    df['predict_from_withstop'] = df['predict_from_withstop'].apply(extract_first_string)
    score = train(df, name)
    results[name] = [score]  # Store score as a list

    print(score)

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('/content/drive/MyDrive/NAEP_Comp/MathBert+CNN/Results_Cohen_' + str(name) + '.csv', index=False)

VH304954


Some weights of the model checkpoint at tbs17/MathBERT were not used when initializing BertCNNLSTMClassifier: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertCNNLSTMClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertCNNLSTMClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertCNNLSTMClassifier were not initialized from the model checkpoint at tb

KeyboardInterrupt: ignored

In [ ]:
inputs = df.predict_from.values
labels = df.assigned_score.values
print("Train data size ", len(inputs))
print('* Original:  ', inputs[0])
# Print the sentence split into tokens.
print('* Tokenized: ', bert_tokenizer.tokenize(inputs[0]))
# Print the sentence mapped to token ids.
print('* Token IDs: ', bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize(inputs[0])))

In [ ]:
# Set the maximum sequence length.
MAX_LEN = 100

# Print BERTs special PAD token and its index in the vocabulary
print(f'Padding token: "{bert_tokenizer.pad_token}", ID: {bert_tokenizer.pad_token_id}')

In [ ]:
tokenized_inputs = bert_tokenizer(
    inputs.tolist(),          # Input text
    add_special_tokens=True,  # add '[CLS]' and '[SEP]'
    padding='max_length',     # pad to a length specified by the max_length
    max_length=MAX_LEN,       # truncate all sentences longer than max_length
    return_tensors='pt',
    truncation = True    # return everything we need as PyTorch tensors
)

input_ids = tokenized_inputs['input_ids']
attention_masks = tokenized_inputs['attention_mask']

# Print sentence 0, now as a list of IDs.
print('Original: ', tokenized_inputs['input_ids'][0])
print('* Token IDs:', tokenized_inputs['attention_mask'][0])
print('* Tokenized:', bert_tokenizer.decode(tokenized_inputs['input_ids'][0]))
print('* Attention_mask', tokenized_inputs['attention_mask'][0])

In [ ]:
bert_train_dataloader, bert_validation_dataloader = train_valid_split(
    input_ids=input_ids,
    attention_masks=attention_masks,
    labels=labels,
    batch_size=32
)

In [ ]:
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining


In [ ]:
mathbert = BertForSentenceClassification.from_pretrained(
    "tbs17/MathBERT",  # the name of the pretrained model
    num_labels=4,      # the number of classes in our downstream task
)

In [ ]:
# Model parameters visualization
params = list(mathbert.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer Layer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# About 2-3 seconds per epoch using GPU
mathbert_loss_vals, mathbert_eval_accs = train_model(
    model=mathbert,
    epochs=1,
    train_dataloader=bert_train_dataloader,
    validation_dataloader=bert_validation_dataloader
)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

def plot_loss_and_acc(loss_vals, eval_accs):
    sns.set(style='darkgrid')
    sns.set(font_scale=1.5)
    plt.rcParams["figure.figsize"] = (12,6)
    fig, ax1 = plt.subplots(1,1)
    ax1.plot(loss_vals, 'b-o', label = 'training loss')
    ax2 = ax1.twinx()
    ax2.plot(eval_accs, 'y-o', label = 'validation accuracy')
    ax2.set_title("Training loss and validation accuracy")
    ax2.set_xlabel("Epoch")
    ax1.set_ylabel("Loss", color='b')
    ax2.set_ylabel("Accuracy", color='y')
    ax1.tick_params(axis='y', rotation=0, labelcolor='b' )
    ax2.tick_params(axis='y', rotation=0, labelcolor='y' )
    plt.show()

In [ ]:
plot_loss_and_acc(mathbert_loss_vals, mathbert_eval_accs)

In [ ]:
mathbert_frozen = BertForSentenceClassification.from_pretrained(
    "tbs17/MathBERT",  # the name of the pretrained model
    num_labels=4,      # the number of classes in our downstream task
)

In [ ]:
for name, param in mathbert_frozen.named_parameters():
	# Only compute gradients for parameters of our
	# newly added classifier. BERT will not be trained.
	if 'classifier' not in name:
		param.requires_grad = False

In [ ]:
# About 1 second per epoch on GPU
mathbert_frozen_loss_vals, mathbert_frozen_eval_accs = train_model(
    model=mathbert_frozen,
    epochs=3,
    train_dataloader=bert_train_dataloader,
    validation_dataloader=bert_validation_dataloader
)